**DATA612 Project2**

**Gullit Navarrete**

**6/15/25**

**Introduction**

**Data Import**

The types of recommender systems that I plan to build are for my "Dataset3JokeSet" .csv file, from https://eigentaste.berkeley.edu/dataset/ Jester Dataset 3, Content-Based Filtering and Item-Item Collaborative Filtering. First, I'll read the 150 texts of Jokes into jokes_df.

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/GullitNa/DATA612-Project2/main/Dataset3JokeSet.csv"
jokes_df = pd.read_csv(url, header=None, encoding='ISO-8859-1')
jokes_df.columns = ['Joke']
jokes_df.head()

,Joke
0,"A man visits the doctor. The doctor says ""I ha..."
1,This couple had an excellent relationship goin...
2,Q. What's 200 feet long and has 4 teeth? A. ...
3,Q. What's the difference between a man and a t...
4,Q.\tWhat's O. J. Simpson's Internet address? ...


After importing the dataset using my Github Repository, I aim to add the 150 user ratings but I can't add it to my github repository due to its huge size followed by this message when uploading: "Yowza, that’s a big file. Try again with a file smaller than 25MB." To work around this, I intend to break apart the "FINAL jester 2006-15.csv" into seperate parts and then upload into github from there to eventually merge.

In [7]:
df = pd.read_csv("/Dataset3jester_ratings.csv", header=None)

df.iloc[:20000].to_csv("jester_part1.csv", index=False, header=False)
df.iloc[20000:40000].to_csv("jester_part2.csv", index=False, header=False)
df.iloc[40000:].to_csv("jester_part3.csv", index=False, header=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-3500932331>", line 1, in <cell line: 0>
    df = pd.read_csv("/Dataset3jester_ratings.csv", header=None)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

TypeError: object of type 'NoneType' has no len()

After downloading these into my PC, followed up by uploading these 3 broken-up parts into Github repository, I then read them into their respective dataframes and then merge the dataframes into ratings_df.

In [2]:
url1 = "https://raw.githubusercontent.com/GullitNa/DATA612-Project2/main/jester_part1.csv"
url2 = "https://raw.githubusercontent.com/GullitNa/DATA612-Project2/main/jester_part2.csv"
url3 = "https://raw.githubusercontent.com/GullitNa/DATA612-Project2/main/jester_part3.csv"

df1 = pd.read_csv(url1, header=None)
df2 = pd.read_csv(url2, header=None)
df3 = pd.read_csv(url3, header=None)

ratings_df = pd.concat([df1, df2, df3], ignore_index=True)
ratings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,62,99,99,99,99,0.21875,99,-9.28125,-9.28125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
1,34,99,99,99,99,-9.68750,99,9.93750,9.53125,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
2,18,99,99,99,99,-9.84375,99,-9.84375,-7.21875,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
3,82,99,99,99,99,6.90625,99,4.75000,-5.90625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
4,27,99,99,99,99,-0.03125,99,-9.09375,-0.40625,99,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0


**Data Cleaning**

First off, the ratings dataset should be cleaned by dropping the first column which is simply just the # of the Joke, not needed for our recommender system building. Since 99 means that the rating is null, I might as well change it to NaN to use for streamlined code later, and then I should load and align the jokes with their respective ratings.

In [3]:
ratings_df = ratings_df.drop(columns=0)
ratings_df.replace(99.0, np.nan, inplace=True)

In [4]:
jokes_url="https://raw.githubusercontent.com/GullitNa/DATA612-Project2/main/Dataset3JokeSet.csv"
jokes_df = pd.read_csv(jokes_url, header=None, encoding='ISO-8859-1')
jokes_df.columns = ['Joke']

**Content-Based Filtering**

For my first recommender system, I approach using content-based filtering, which in this particular case, the system would recommend jokes based on the textual similarity between them. My idea is to use the joke content itself to find other jokes with similar wording, structure, or topics. This approach does not rely on user ratings, only on the jokes text dataset.

To implement this, I used "TfidfVectorizer" to convert each joke into a vector of weighted terms using Term Frequency-Inverse Document Frequency (TF-IDF). Then, I applied cosine similarity to calculate how similar each joke vector is to all others.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words='english')
# Create TF-IDF matrix where each row = joke, and each column= word
tfidf_matrix = vectorizer.fit_transform(jokes_df['Joke'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)

TF-IDF matrix shape: (150, 2009)


Afterwards, I intend to make code that functions to pairwise cosine similarity between all jokes in the dataset using their TF-IDF vector representations. Each entry is suppose to reflect how similar one particular joke is to another joke based on the words they both share. This matrix is later used to retrieve the most similar jokes for any given input joke.

In [21]:
# Cosine similarity of all joke vectors against each other
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend similar jokes based on TF-IDF cosine similarity
def recommend_similar_jokes(joke_id, top_n=5):
    print(f"\nOriginal Joke [{joke_id}]:\n{jokes_df.iloc[joke_id]['Joke']}\n")

    sim_scores = list(enumerate(cosine_sim[joke_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]

    print(f"Top {top_n} similar jokes:")
    for idx, score in sim_scores:
        print(f"\n[Joke {idx} | Similarity Score: {score:.3f}]")
        print(jokes_df.iloc[idx]['Joke'])
recommend_similar_jokes(joke_id=53, top_n=3)


Original Joke [53]:
The Pope dies and, naturally, goes to heaven. He's met by the reception committee, and after a whirlwind tour he is told that he can enjoy any of the myriad of recreations available. He decides that he wants to read all of the ancient original text of the Holy Scriptures, so he spends the next eon or so learning languages. After becoming a linguistic master, he sits down in the library and begins to pour over every version of the Bible, working back from most recent "Easy Reading" to the original script. All of a sudden there is a scream in the library. The Angels come running in only to find the Pope huddled in his chair, crying to himself and muttering, "An 'R'! The scribes left out the 'R'."  A particularly concerned Angel takes him aside, offering comfort, asks him what the problem is and what does he mean.  After collecting his wits, the Pope sobs again, "It's the letter 'R'. They left out the 'R'. The word was supposed to be CELEBRATE!"

Top 3 similar jokes:


**Item-Item Collaborative Filtering**

Now for this approach, I plan to use item-item collaborative filtering to identify jokes that are rated similarly by users. By comparing user rating patterns across jokes, I can then set it up to find jokes with similar audience reception regardless of their content.

In [24]:
item_user_matrix = ratings_df.T
# Item-item similarity based on user ratings
item_similarity = cosine_similarity(item_user_matrix.fillna(0))
def recommend_similar_items(joke_id, top_n=5):
    print(f"\nOriginal Joke [{joke_id}]:\n{jokes_df.iloc[joke_id]['Joke']}\n")
    sim_scores = list(enumerate(item_similarity[joke_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    print(f"Top {top_n} similar jokes:")
    for idx, score in sim_scores:
        print(f"\n[Joke {idx} | Similarity Score: {score:.3f}]")
        print(jokes_df.iloc[idx]['Joke'])
recommend_similar_items(joke_id=53, top_n=3)


Original Joke [53]:
The Pope dies and, naturally, goes to heaven. He's met by the reception committee, and after a whirlwind tour he is told that he can enjoy any of the myriad of recreations available. He decides that he wants to read all of the ancient original text of the Holy Scriptures, so he spends the next eon or so learning languages. After becoming a linguistic master, he sits down in the library and begins to pour over every version of the Bible, working back from most recent "Easy Reading" to the original script. All of a sudden there is a scream in the library. The Angels come running in only to find the Pope huddled in his chair, crying to himself and muttering, "An 'R'! The scribes left out the 'R'."  A particularly concerned Angel takes him aside, offering comfort, asks him what the problem is and what does he mean.  After collecting his wits, the Pope sobs again, "It's the letter 'R'. They left out the 'R'. The word was supposed to be CELEBRATE!"

Top 3 similar jokes:


**Evaluation: Comparing and Contrasting**

From building the content-based filtering and item-item collaborative filtering systems, the approaches are both relatively similar in building and very different in approach and result. Content-based filtering recommends items based on the features of the items themselves, in this case, the joke text. It analyzes the words used in the jokes and finds similar jokes based on textual patterns using techniques like TF-IDF and cosine similarity. In contrast, item-item collaborative filtering focuses on user behavior, recommending jokes that received similar ratings from users, even if their content is unrelated, basically acting as showing other highly rated Jokes that are popular in that dataset among those rated users. While content-based filtering is appropriate if my goal was to recognize worded similarities, it doesn't consider audience preferences. On the other hand, item-item collaborative filtering does capture shared user ratings, but its accuracy depends heavily on having a sufficient number of user ratings.

**Conclusion**

Working with a different dataset than last time, JesterDataset3 https://eigentaste.berkeley.edu/dataset/, I worked on user ratings for 150 jokes and their joke texts in a seperate dataset. The dataset was split, cleaned, and loaded from GitHub to ensure compatibility with Google Colab. I implemented two recommender systems: content-based filtering using TF-IDF and cosine similarity on joke text, and item-item collaborative filtering using user rating patterns. While content-based filtering was able to identify jokes with similar language and/or structure, the item-item collaborative approach gave recommendations based on actual user preferences from their ratings. For a dataset like this, which includes thousands of user ratings for a smaller set of 150 jokes, item-item collaborative filtering is the more effective approach because it captures audience preference beyond the surface-level wording of user-based collaborative filtering.